<a href="https://colab.research.google.com/github/unnatihasija1/HANA-Cloud-ML/blob/main/HANA_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install hana_ml

import hana_ml
import pandas as pd
df_data = pd.read_csv('bank-full.csv', sep = ';')
df_data.head(5)
df_data.columns = map(str.upper, df_data.columns)

In [ ]:
df_data = df_data.drop(['DURATION'],
                       axis = 1)
df_data = df_data.rename(index = str, columns = {'Y': 'PURCHASE'})
df_data.insert(0, 'CUSTOMER_ID', df_data.reset_index().index)
df_data.head(5)

In [ ]:
import hana_ml.dataframe as dataframe

# Instantiate connection object
conn = dataframe.ConnectionContext(address = '<hostname>'
                                   port = 443, 
                                   user = 'UNNATI', 
                                   password = '<userpassword>', 
                                   encrypt = 'true',
                                   sslValidateCertificate = 'false' 
                                  )

# Send basic SELECT statement and display the result
sql = 'SELECT 12345 FROM DUMMY'
df_remote = conn.sql(sql)
print(df_remote.collect())

In [ ]:
C:\Program Files\SAP\hdbclient>hdbuserstore -i SET MYHANACLOUD "hostname:portnumber"
conn = dataframe.ConnectionContext(userkey = '<keyname>',
                                   encrypt = 'true')

In [ ]:
df_remote = dataframe.create_dataframe_from_pandas(connection_context = conn, 
                                                   pandas_df = df_data, 
                                                   table_name = 'BANKMARKETING',
                                                   force = True,
                                                   replace = False)

In [ ]:
df_topredict = pd.DataFrame(data = None, 
                            columns = df_data.columns.drop('PURCHASE'))
for xx in df_topredict.columns:
    df_topredict[xx] = df_topredict[xx].astype(df_data[xx].dtypes.name)

In [ ]:
df_topredict = pd.concat([df_topredict, pd.DataFrame({'CUSTOMER_ID': 1, 
                                    'AGE': 40, 
                                    'JOB': 'entrepreneur', 
                                    'MARITAL': 'married', 
                                    'EDUCATION': 'secondary',  
                                    'DEFAULT': 'no', 
                                    'BALANCE': 3000,
                                    'HOUSING': 'yes',
                                    'LOAN': 'no',
                                    'CONTACT': 'unknown',
                                    'DAY': 10,
                                    'MONTH': 'may',
                                    'CAMPAIGN': 1,
                                    'PDAYS': -1,
                                    'PREVIOUS': 0,
                                    'POUTCOME': 'unknown'}, index=[0])])
df_topredict = pd.concat([df_topredict, pd.DataFrame({'CUSTOMER_ID': 2, 
                                    'AGE': 65, 
                                    'JOB': 'management', 
                                    'MARITAL': 'single', 
                                    'EDUCATION': 'tertiary',  
                                    'DEFAULT': 'no', 
                                    'BALANCE': 3000,
                                    'HOUSING': 'no',
                                    'LOAN': 'no',
                                    'CONTACT': 'telephone',
                                    'DAY': 10,
                                    'MONTH': 'mar',
                                    'CAMPAIGN': 1,
                                    'PDAYS': -1,
                                    'PREVIOUS': 12,
                                    'POUTCOME': 'success'}, index=[0])])

In [ ]:
df_remote = dataframe.create_dataframe_from_pandas(connection_context = conn, 
                                                   pandas_df = df_topredict, 
                                                   table_name = 'BANKMARKETING_TOPREDICT',
                                                   force = True,
                                                   replace = False)

In [ ]:
df_remote = conn.table(table = 'BANKMARKETING', schema = 'UNNATI').sort('CUSTOMER_ID', desc = False)
df_remote.head(5).collect()

In [ ]:
df_remote.count()
df_remote.dtypes()
df_remote.describe().collect()

In [ ]:
top_n = 5 
variable_name = 'PURCHASE'

# Get Top N categories
total_count = df_remote.count()
df_remote_col_frequency = df_remote.agg([('count', variable_name, 'COUNT')],  group_by = variable_name)
df_col_frequency = df_remote_col_frequency.sort("COUNT", desc = True).head(top_n).collect()
df_col_frequency['PERCENT'] = round(df_col_frequency['COUNT'] / total_count, 2)
df_col_frequency.style.format({'COUNT':'{0:,.0f}', 'PERCENT':'{0:,.1%}'})

In [ ]:
%matplotlib inline 
df_col_frequency.plot.bar(x = 'PURCHASE', y = 'COUNT', title = 'Top ' + str(top_n));

In [ ]:
import matplotlib.pyplot as plt
from hana_ml.visualizers.eda import EDAVisualizer
f = plt.figure()
ax1 = f.add_subplot(111) # 111 refers to 1x1 grid, 1st subplot
eda = EDAVisualizer(ax1)
ax, cont = eda.box_plot(data = df_remote, column = 'BALANCE', groupby = 'PURCHASE', outliers = True)

In [ ]:
from hana_ml.algorithms.apl.gradient_boosting_classification import GradientBoostingBinaryClassifier
gbapl_model = GradientBoostingBinaryClassifier()
col_target = 'PURCHASE'
target_value = 'yes'
col_id = 'CUSTOMER_ID'

In [ ]:
from hana_ml.algorithms.apl.gradient_boosting_classification import GradientBoostingBinaryClassifier
gbapl_model = GradientBoostingBinaryClassifier()
col_target = 'PURCHASE'
target_value = 'yes'
col_id = 'CUSTOMER_ID'